Quickstart
=====

In [ ]:
%matplotlib inline
import os
import tempfile

import scipy.stats as st

from pyabc import (ABCSMC, RV, Distribution,
                   PercentileDistanceFunction,
                   ConstantPopulationStrategy)

# Define a gaussian model
sigma = .5


def model(parameters):
    # sample from a gaussian
    y = st.norm(parameters.x, sigma).rvs()
    # return the sample as dictionary
    return {"y": y}

In [ ]:
# We define two models, but they are identical so far
models = [model, model]


# However, our models' priors are not the same. Their mean differs.
mu_x_1, mu_x_2 = 0, 1
parameter_priors = [
    Distribution(x=RV("norm", mu_x_1, sigma)),
    Distribution(x=RV("norm", mu_x_2, sigma))
]

In [ ]:
# We plug all the ABC setup together with 2 populations maximum
population_strategy = ConstantPopulationStrategy(100, 2)
abc = ABCSMC(models, parameter_priors,
             PercentileDistanceFunction(measures_to_use=["y"]),
             population_strategy)

In [ ]:
# y_observed is the important piece here: our actual observation.
y_observed = 1
# Finally we add meta data such as model names
# and define where to store the results
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))
abc.set_data({"y": y_observed}, db_path)

In [ ]:
# We run the ABC
minimum_epsilon = .05
history = abc.run(minimum_epsilon)

# Evaluate the model probabililties
model_probabilities = history.get_model_probabilities(history.max_t)
model_probabilities

In [ ]:
model_probabilities.plot.bar();